In [1]:
%load_ext autodisplay
_autodisplay.display_line = False

import random
random.seed(8)

from functools import partial

import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D

from jim import Vector, Body, Universe, transpose_states, lorentz_force, implement_field

In [2]:
def circular_field(r):
    return Vector(0, 0, 1) @ (r - r[2] * Vector(0, 0, 1)).unit

In [3]:
circular_field(Vector(1, 0, 0))

Vector(0.0, 1.0, 0.0)

In [4]:
body = Body(Vector(5, 0, 0), Vector(0, 2, 0), None, 1)
body.q = 2

In [5]:
universe = Universe(Vector(0, 0, 0))
universe.add(body)
implement_field(
    universe,
    partial(lorentz_force, field=lambda r: (Vector.null_vector(3), circular_field(r)))
)

----

In [6]:
dt = 1. / 30
frames = 1800

In [7]:
# set up figure and animation
fig = plt.figure(figsize=(12.80, 7.20))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(
    121,
    aspect="equal",
    autoscale_on=False,
    projection="3d",
)
ax.grid()

ax.set_xlim3d(-15, 15)
ax.set_ylim3d(-15, 15)
ax.set_zlim3d(-15, 15)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

ax2d = fig.add_subplot(
    122,
    aspect="equal",
)
ax2d.grid()
ax2d.set_xlim(0, 20)
ax2d.set_ylim(-3, 3)
ax2d.set_xlabel(r"$\frac{t}{s}$")
ax2d.set_ylabel(r"$\frac{v}{\frac{m}{s}}$")

Ein roter Punkt im Ursprung zur Orientierung:

In [8]:
ax.scatter([0], [0], c="red");

In [9]:
# particle holds the location of the particle
particle, = ax.plot([], [], 'bo', ms=10)
trace, = ax.plot([], [], "-")
trace_data = [[] for _ in range(3)]

v_traces = [
    ax2d.plot([], [], label="$v_{}$".format(label))[0]
    for label in "xyz"
]
v_data = [[] for _ in range(3)]

v_norm, = ax2d.plot([], [], label=r"$|\vec v|$")
v_norm_data = []

time_data = []

In [10]:
def init():
    """initialize animation"""
    particle.set_data([], [])
    particle.set_3d_properties([])
    trace.set_data([], [])
    trace.set_3d_properties([])
    
    for v_trace in v_traces:
        v_trace.set_data([], [])
    
    v_norm.set_data([], [])
    
    return (particle, trace, *v_traces, v_norm)

In [11]:
def exhaust(iterator):
    for _ in iterator:
        pass

In [12]:
def animate(i):
    """perform animation step"""
    exhaust(universe.simulate(dt, 0.0005))
    time_data.append(universe.time)
    state = universe.state()[0]
    x, y, z = state.r
    
    v_norm_data.append(state.v.norm)
    
    particle.set_data(x, y)
    particle.set_3d_properties(z)
    
    for coord, coord_trace in zip(state.r, trace_data):
        coord_trace.append(coord)
    
    trace.set_data(*trace_data[:2])
    trace.set_3d_properties(trace_data[2])
    
    for v_coord, v_trace_data, v_trace in zip(state.v, v_data, v_traces):
        v_trace_data.append(v_coord)
        v_trace.set_data(time_data, v_trace_data)
    
    v_norm.set_data(time_data, v_norm_data)

    if not i % 10:
        field = circular_field(state.r)
        particle.set_label(r"$\vec B(\vec r) = ({:.2f}, {:.2f}, {:.2f})$".format(*field))
        ax.legend(loc=1)
        ax2d.legend(loc=1)
    
    if universe.time > 15:
        ax2d.set_xlim(universe.time - 15, universe.time + 5)
    
    return (particle, trace, *v_traces, v_norm)

In [13]:
ani = animation.FuncAnimation(
    fig,
    animate,
    frames=frames,
    interval=10,
    blit=True,
    init_func=init
)

In [14]:
# save the animation as an mp4.
ani.save('curvature_drift.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

In [15]:
# Uncomment to see a live and interactive view of the system.
#plt.show()

$v_R$ bezeichne die mittlere Driftgeschwindigkeit des Führungszentrums, die durch Krümmungsdrift hervorgerufen wird.

In [16]:
v_R_simulated >>= body.r[2] / universe.time

`v_R_simulated` at line 1:


0.36420747519602453

In [17]:
v_R_expected >>= body.m / body.q * (body.v * circular_field(body.r))**2 / 5

`v_R_expected` at line 1:


0.3942310945510432

Ist die Abweichung zwischen `v_R_simulated` und `v_R_expected` auf die Simulation zurückzuführen? 

In [18]:
v_data[2][-20:]

[0.5582132223830619,
 0.5350865073769898,
 0.5111270430032246,
 0.48644940518880075,
 0.4611717116318749,
 0.435415065701715,
 0.40930298438844753,
 0.3829608128286939,
 0.3565151280245391,
 0.3300931344568664,
 0.30382205436929943,
 0.277828515565007,
 0.25223793961459173,
 0.22717393341821576,
 0.20275768709794775,
 0.1791073812159408,
 0.15633760631925717,
 0.13455879780174973,
 0.11387668904618393,
 0.09439178576458147]

Es wurde *fast* eine ganze Anzahl an Perioden simuliert, daher kann in guter Näherung über alle `v_data[2]` (enspricht $\frac{\Delta r_z}{\Delta t}$) gemittelt werden und es muss nicht die letzte (unvollständige) Periode ignoriert werden.